In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class PCA:
    """
    Principal Component Analysis (PCA) implementation

    PCA finds the directions (principal components) of maximum variance
    in the data and project the data onto these directions
    """

    def __init__(self, n_components=None):
        """
        Initialize PCA

        Args:
            n_components: Number of principal components to keep
            if None, keeps all components 
        """

        self.n_components = n_components
        self.mean = None # mean of each feature (for centering)
        self.components_ = None # principal components (eigenvectors)
        self.explained_variance = None # variance explained by each component
        self.explained_variance_ration_ = None # percentage of variance explained

    def fit(self, X):
        """
        Fit PCA model to the data

        Steps:
            1 - center the data (subtract mean)
            2 - compute covariance matrix
            3 - find eigenvectors and eigenvalues (principal components)
            4 - sort components by explained variance
            5 - select top n_components

        Args:
            X: input data, shape(n_samples, n_features)
        """
        n_samples, n_features = X.shape

        # step 1, why? because PCA finds directions of variance. The mean doesn't affect variance,
        # but centering ensures the first principal component goes through the origin
        self.mean = np.mean(X, axis  = 0) # mean of each feature
        X_centered = X - self.mean

        # step 2, why? because the covariance matrix captures relationships between features
        # the (i,j) entry shows covariance between feature i and feature j
        # we want to find directions that maximize covariance structure
        covariance_matrix = np.cov(X_centered, rowvar=False)
        # rowvar = false means each column is a variable (feature)
        # Shape: (n_features, n_features)

        # alternative: you could compute X_centered.T @ X_centered / (n_samples -1)
        # this is mathematically equivalent but computationally different

        # step 3, why? Because eigenvectors of covariance matrix  = principal components
        # eigenvalues = variance explained by each component
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
        # eigh() is for symmetric matrices (covariance is symmetric)
        # returns eigenvalues and eigenvectors sorted in ascending order

        # step 4, why? we want components with highest variance first
        idx = np.argsort(eigenvalues)[::-1] # indices for descending sort
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]

        # step 5
        self.components_ = eigenvectors
        self.explained_variance = eigenvalues
        self.explained_variance_ration_ = eigenvalues / eigenvalues.sum()

        # if n_components specified, truncate 
        if self.n_components is not None:
            self.components_ = self.components_[:, :self.n_components]
            self.explained_variance = self.explained_variance[:, :self.n_components]
            self.explained_variance_ration_ = self.explained_variance_ration_[:, :self.n_components]
        return self
    
    def transform(self, X):
        """
        Transform data to principal component space 

        Project data onto principal components: Y = X_centered * W
        where W is matrix of principal components

        Args:
             X: input data, shape (n_samples, n_features)

        returns:
            X_transformed: Projected data, shape(n_samples, n_components)
        """

        # center the data using stored mean
        X_centered = X - self.mean

        # Project onto principal components
        # Each column of components_ is a principal component(eigenvector)
        # Dot product projects each sample onto each component
        X_transformed = X_centered @ self.components_

        return X_transformed
     
    def fit_transform(self, X):
        """
        Fit PCA and transform data in one step
        """
        self.fit(X)
        return self.transform(X)
    
    def inverse_transform(self, X_transformed):
        """
        Transform data back to original space
        Reconstruct data from principal components: X_reconstructed = Y * W ^T + mean

        Args:
            X_transformed: Data in PCA space, shape (n_samples, n_components)

        returns:
            X_reconstructured: Data in original feature space
        """

        # project back to original space: Y * w^t
        X_reconstructed = X_transformed @ self.components_.T

        # add back the mena
        X_reconstructed = X_reconstructed + self.mean

        return X_reconstructed
    def get_cumulative_variance(self):
        """
        Get cumulative explained variance ratio

        Useful for deciding how many components to keep
        """
        return np.cumsum(self.explained_variance_ration_)
    
    def visualize_pca():
        """
        Create a simple 2D example to visualize PCA
        """
        # Generate correlated 2D data
        np.random.seed(42)
        n_samples = 200
    
        # Create data with strong correlation (elongated ellipsoid)
        mean = [2, 3]
        cov = [[2, 1.8], [1.8, 2]]  # Covariance matrix
        X = np.random.multivariate_normal(mean, cov, n_samples)
    
        # Fit PCA with 2 components (we'll use both for visualization)
        pca = PCA(n_components=2)
        X_transformed = pca.fit_transform(X)
    
        # Plot original data and principal components
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
        # Plot 1: Original data
        axes[0].scatter(X[:, 0], X[:, 1], alpha=0.6)
        axes[0].scatter(pca.mean_[0], pca.mean_[1], color='red', s=100, label='Mean')
    
        # Draw principal components as arrows from the mean
        scale = 3  # Scale factor for arrows
        for i in range(2):
            component = pca.components_[:, i]
            axes[0].arrow(pca.mean_[0], pca.mean_[1], 
                     component[0] * scale * np.sqrt(pca.explained_variance_[i]),
                     component[1] * scale * np.sqrt(pca.explained_variance_[i]),
                     color='red', width=0.05, head_width=0.3, alpha=0.8,
                     label=f'PC{i+1}' if i == 0 else "")
    
        axes[0].set_xlabel('Feature 1')
        axes[0].set_ylabel('Feature 2')
        axes[0].set_title('Original Data with Principal Components')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        axes[0].axis('equal')
    
        # Plot 2: Transformed data (in PCA space)
        axes[1].scatter(X_transformed[:, 0], X_transformed[:, 1], alpha=0.6)
        axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
        axes[1].set_xlabel('Principal Component 1')
        axes[1].set_ylabel('Principal Component 2')
        axes[1].set_title('Data in PCA Space (PC1 vs PC2)')
        axes[1].grid(True, alpha=0.3)
        axes[1].axis('equal')
    
        # Plot 3: Variance explained
        axes[2].bar(range(1, 3), pca.explained_variance_ratio_ * 100, alpha=0.7)
        axes[2].set_xlabel('Principal Component')
        axes[2].set_ylabel('Variance Explained (%)')
        axes[2].set_title('Variance Explained by Each Component')
        axes[2].set_xticks([1, 2])
        axes[2].grid(True, alpha=0.3, axis='y')
    
        plt.tight_layout()
        plt.show()
    
        # Print statistics
        print("="*60)
        print("PCA RESULTS:")
        print("="*60)
        print(f"Original data shape: {X.shape}")
        print(f"Transformed data shape: {X_transformed.shape}")
        print(f"\nPrincipal Components (eigenvectors):")
        print(f"PC1: {pca.components_[:, 0]}")
        print(f"PC2: {pca.components_[:, 1]}")
        print(f"\nExplained Variance:")
        print(f"PC1: {pca.explained_variance_[0]:.3f} ({pca.explained_variance_ratio_[0]*100:.1f}%)")
        print(f"PC2: {pca.explained_variance_[1]:.3f} ({pca.explained_variance_ratio_[1]*100:.1f}%)")
        print(f"\nTotal variance in original data: {np.sum(np.var(X, axis=0)):.3f}")
        print(f"Total variance captured by PCA: {np.sum(pca.explained_variance_):.3f}")
        
        # Demonstrate reconstruction
        print("\n" + "="*60)
        print("RECONSTRUCTION DEMONSTRATION:")
        print("="*60)
        
        # Take one sample
        sample_idx = 0
        original_sample = X[sample_idx]
        
        # Transform to PCA space (1 component only for compression demo)
        pca_1d = PCA(n_components=1)
        X_1d = pca_1d.fit_transform(X)
        
        # Reconstruct from 1D representation
        reconstructed_1d = pca_1d.inverse_transform(X_1d[sample_idx].reshape(1, -1))
        
        print(f"Original sample: {original_sample}")
        print(f"Transformed to 1D (PC1 value): {X_1d[sample_idx][0]:.3f}")
        print(f"Reconstructed from 1D: {reconstructed_1d[0]}")
        print(f"Reconstruction error (1 component): {np.linalg.norm(original_sample - reconstructed_1d[0]):.3f}")
        
        # Compare with 2-component reconstruction (should be perfect)
        reconstructed_2d = pca.inverse_transform(X_transformed[sample_idx].reshape(1, -1))
        print(f"\nReconstruction error (2 components): {np.linalg.norm(original_sample - reconstructed_2d[0]):.3f}")
        print("(Should be ~0 since we kept all components)")

    # Run the visualization
    visualize_pca()